In [1]:
import pandas as pd

citibike_data = None

for month in range(1,13):
    if month < 10:
        month_str = f'0{month}'
    else:
        month_str = f'{month}'
    print('Loading month', month_str)
    current_month = pd.read_csv(f'../dataset/2018{month_str}-citibike-tripdata.csv')
    
    if citibike_data is None:
        citibike_data = current_month
    else:
        citibike_data = citibike_data.append(current_month)
citibike_data = citibike_data.dropna()

Loading month 01
Loading month 02
Loading month 03
Loading month 04
Loading month 05
Loading month 06
Loading month 07
Loading month 08
Loading month 09
Loading month 10
Loading month 11
Loading month 12


In [2]:
citibike_data = citibike_data.dropna()

import numpy as np

citibike_data['starttime'] = pd.to_datetime(citibike_data['starttime'])
citibike_data['stoptime'] = pd.to_datetime(citibike_data['stoptime'])

citibike_data['tripduration'] = (citibike_data['stoptime'] - citibike_data['starttime']).dt.total_seconds().apply(np.floor).astype(int)

filter_columns = [
    'tripduration', 
    'starttime', 
    'stoptime', 
    'start station id',
    'start station latitude',
    'start station longitude',
    'end station id',
    'end station latitude',
    'end station longitude',
    'usertype',
    'birth year',
    'gender'
]
citibike_data = citibike_data[filter_columns]
citibike_data = citibike_data[citibike_data['tripduration'] <= 20*24*60*60]

citibike_data = citibike_data[citibike_data['end station latitude'] <= 41.13]
citibike_data = citibike_data[citibike_data['start station latitude'] <= 41.13]
citibike_data = citibike_data[citibike_data['end station latitude'] >= 40.12]
citibike_data = citibike_data[citibike_data['start station latitude'] >= 40.12]
citibike_data = citibike_data[citibike_data['end station longitude'] >= -74.40]
citibike_data = citibike_data[citibike_data['start station longitude'] >= -74.40]
citibike_data = citibike_data[citibike_data['end station longitude'] <= -73.19]
citibike_data = citibike_data[citibike_data['start station longitude'] <= -73.19]

citibike_data['start_weekday'] = citibike_data['starttime'].dt.day_name()

citibike_data

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday
0,970,2018-01-01 13:50:57.434,2018-01-01 14:07:08.186,72.0,40.767272,-73.993929,505.0,40.749013,-73.988484,Subscriber,1992,1,Monday
1,723,2018-01-01 15:33:30.182,2018-01-01 15:45:33.341,72.0,40.767272,-73.993929,3255.0,40.750585,-73.994685,Subscriber,1969,1,Monday
2,496,2018-01-01 15:39:18.337,2018-01-01 15:47:35.172,72.0,40.767272,-73.993929,525.0,40.755942,-74.002116,Subscriber,1956,1,Monday
3,306,2018-01-01 15:40:13.372,2018-01-01 15:45:20.191,72.0,40.767272,-73.993929,447.0,40.763707,-73.985162,Subscriber,1974,1,Monday
4,306,2018-01-01 18:14:51.568,2018-01-01 18:19:57.642,72.0,40.767272,-73.993929,3356.0,40.774667,-73.984706,Subscriber,1992,1,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016500,734,2018-12-31 23:55:06.991,2019-01-01 00:07:21.230,336.0,40.730477,-73.999061,379.0,40.749156,-73.991600,Subscriber,1986,1,Monday
1016501,2551,2018-12-31 23:55:44.307,2019-01-01 00:38:15.782,3320.0,40.794067,-73.962868,2006.0,40.765909,-73.976342,Customer,1969,0,Monday
1016502,2376,2018-12-31 23:56:52.850,2019-01-01 00:36:29.286,427.0,40.701907,-74.013942,259.0,40.701221,-74.012342,Subscriber,1979,1,Monday
1016503,2758,2018-12-31 23:58:29.212,2019-01-01 00:44:27.517,3320.0,40.794067,-73.962868,281.0,40.764397,-73.973715,Customer,1969,0,Monday


In [3]:
citibike_data = citibike_data.groupby('usertype').apply(lambda x: x.sample(500000))

In [4]:
citibike_data = citibike_data.reset_index(level=[0,1], drop=True)

In [5]:
features_df = pd.DataFrame(citibike_data)

In [6]:
from sklearn import preprocessing

unique_labels = features_df['usertype'].unique()
unique_weekdays = features_df['start_weekday'].unique()
le_ut = preprocessing.LabelEncoder()
le_ut.fit(unique_labels)
le_wd = preprocessing.LabelEncoder()
le_wd.fit(unique_weekdays)

LabelEncoder()

In [7]:
features_df['usertype'] = le_ut.transform(features_df['usertype']) 
features_df['start_weekday'] = le_wd.transform(features_df['start_weekday']) 

In [8]:
feature_columns = [
    'tripduration', 
    'start station latitude',
    'start station longitude',
    'end station latitude',
    'end station longitude',
    'birth year',
    'start_weekday',
    'gender'
]
label_column = 'usertype'

features_df = features_df[feature_columns + [label_column]]
features_df

,tripduration,start station latitude,start station longitude,end station latitude,end station longitude,birth year,start_weekday,gender,usertype
0,58303,40.702772,-73.993836,40.702772,-73.993836,1969,0,0,0
1,1593,40.717746,-73.956001,40.680983,-73.950048,1969,3,0,0
2,1186,40.766368,-73.971518,40.751873,-73.977706,1990,3,1,0
3,1602,40.665147,-73.976376,40.665147,-73.976376,1969,6,0,0
4,320,40.700469,-73.991454,40.702772,-73.993836,1969,2,0,0
...,...,...,...,...,...,...,...,...,...
999995,564,40.739355,-73.999318,40.746201,-73.988557,1971,6,2,1
999996,414,40.721101,-73.991925,40.719105,-73.999733,1993,6,2,1
999997,1404,40.693830,-73.990539,40.707645,-73.968415,1992,5,2,1
999998,2293,40.746745,-74.007756,40.692216,-73.984284,1987,0,2,1


In [9]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(
    features_df, 
    test_size=0.33, 
    random_state=42, 
    stratify=features_df[label_column]
)

In [10]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(
    train_set[feature_columns], 
    train_set[label_column]
)

In [11]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.8057902705870861
[0 1]
['Customer' 'Subscriber']
              precision    recall  f1-score   support

           0       0.80      0.81      0.81    165000
           1       0.81      0.80      0.81    165000

    accuracy                           0.81    330000
   macro avg       0.81      0.81      0.81    330000
weighted avg       0.81      0.81      0.81    330000



In [12]:
clf.feature_importances_

array([0.1864002 , 0.0744717 , 0.07220274, 0.07518964, 0.07263473,
       0.0748056 , 0.03192732, 0.41236807])

In [13]:
unique_labels

array(['Customer', 'Subscriber'], dtype=object)

In [14]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_set[feature_columns], train_set[label_column])

print(clf.feature_importances_)

[0.2673286  0.0248018  0.02623263 0.01415149 0.01068899 0.20425404
 0.03513089 0.41741158]


In [15]:
predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.8262105090155288
[0 1]
['Customer' 'Subscriber']
              precision    recall  f1-score   support

           0       0.92      0.66      0.77    165000
           1       0.73      0.94      0.83    165000

    accuracy                           0.80    330000
   macro avg       0.83      0.80      0.80    330000
weighted avg       0.83      0.80      0.80    330000



In [16]:
clf2 = RandomForestClassifier(max_depth=10, random_state=0)
clf2.fit(train_set[feature_columns], train_set[label_column])

print(clf2.feature_importances_)

[0.23615598 0.01245087 0.01057944 0.01275181 0.00999878 0.22013933
 0.03543765 0.46248613]


In [17]:
predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.8262105090155288
[0 1]
['Customer' 'Subscriber']
              precision    recall  f1-score   support

           0       0.92      0.66      0.77    165000
           1       0.73      0.94      0.83    165000

    accuracy                           0.80    330000
   macro avg       0.83      0.80      0.80    330000
weighted avg       0.83      0.80      0.80    330000



In [18]:
features_df = pd.DataFrame(citibike_data)

In [19]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

from sklearn import preprocessing

userttype_unique = features_df[label_column].unique()
le_pp = preprocessing.LabelEncoder()
le_pp.fit(userttype_unique)
usertype_encoded = le_pp.transform(features_df[label_column])
one_hot_encoded_usertype = to_categorical(usertype_encoded)
one_hot_encoded_usertype

Using TensorFlow backend.


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [20]:
usertype_one_hot = label_column + '_one_hot'
features_df[usertype_one_hot] = list(one_hot_encoded_usertype)
features_df

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday,usertype_one_hot
0,58303,2018-03-30 19:21:30.275,2018-03-31 11:33:13.469,217.0,40.702772,-73.993836,217.0,40.702772,-73.993836,0,1969,0,0,"[1.0, 0.0]"
1,1593,2018-08-26 19:06:15.115,2018-08-26 19:32:48.922,3090.0,40.717746,-73.956001,437.0,40.680983,-73.950048,0,1969,0,3,"[1.0, 0.0]"
2,1186,2018-06-24 17:15:54.068,2018-06-24 17:35:40.553,3136.0,40.766368,-73.971518,519.0,40.751873,-73.977706,0,1990,1,3,"[1.0, 0.0]"
3,1602,2018-04-18 19:50:26.512,2018-04-18 20:17:08.947,3300.0,40.665147,-73.976376,3300.0,40.665147,-73.976376,0,1969,0,6,"[1.0, 0.0]"
4,320,2018-05-05 17:34:46.051,2018-05-05 17:40:06.655,407.0,40.700469,-73.991454,217.0,40.702772,-73.993836,0,1969,0,2,"[1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,564,2018-04-11 07:15:59.942,2018-04-11 07:25:24.421,482.0,40.739355,-73.999318,486.0,40.746201,-73.988557,1,1971,2,6,"[0.0, 1.0]"
999996,414,2018-10-31 09:22:41.613,2018-10-31 09:29:36.026,473.0,40.721101,-73.991925,268.0,40.719105,-73.999733,1,1993,2,6,"[0.0, 1.0]"
999997,1404,2018-04-10 18:53:09.395,2018-04-10 19:16:34.059,314.0,40.693830,-73.990539,282.0,40.707645,-73.968415,1,1992,2,5,"[0.0, 1.0]"
999998,2293,2018-07-20 18:12:00.797,2018-07-20 18:50:14.762,459.0,40.746745,-74.007756,390.0,40.692216,-73.984284,1,1987,2,0,"[0.0, 1.0]"


In [21]:
features_df['start_weekday'] = features_df['starttime'].dt.day_name()
features_df['start_weekday_str'] = features_df['start_weekday']
weekday_unique = features_df['start_weekday'].unique()

convert_weekdays = {
    'start_weekday': {
        'Monday': 0, 
        'Tuesday': 1, 
        'Wednesday': 2, 
        'Thursday': 3, 
        'Friday': 4, 
        'Saturday': 5, 
        'Sunday': 6}
}

features_df = features_df.replace(convert_weekdays)
features_df = features_df.drop(['start_weekday_str'], axis=1)

In [22]:
usertype_one_hot = label_column + '_one_hot'
features_df[usertype_one_hot] = list(one_hot_encoded_usertype)
features_df

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday,usertype_one_hot
0,58303,2018-03-30 19:21:30.275,2018-03-31 11:33:13.469,217.0,40.702772,-73.993836,217.0,40.702772,-73.993836,0,1969,0,4,"[1.0, 0.0]"
1,1593,2018-08-26 19:06:15.115,2018-08-26 19:32:48.922,3090.0,40.717746,-73.956001,437.0,40.680983,-73.950048,0,1969,0,6,"[1.0, 0.0]"
2,1186,2018-06-24 17:15:54.068,2018-06-24 17:35:40.553,3136.0,40.766368,-73.971518,519.0,40.751873,-73.977706,0,1990,1,6,"[1.0, 0.0]"
3,1602,2018-04-18 19:50:26.512,2018-04-18 20:17:08.947,3300.0,40.665147,-73.976376,3300.0,40.665147,-73.976376,0,1969,0,2,"[1.0, 0.0]"
4,320,2018-05-05 17:34:46.051,2018-05-05 17:40:06.655,407.0,40.700469,-73.991454,217.0,40.702772,-73.993836,0,1969,0,5,"[1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,564,2018-04-11 07:15:59.942,2018-04-11 07:25:24.421,482.0,40.739355,-73.999318,486.0,40.746201,-73.988557,1,1971,2,2,"[0.0, 1.0]"
999996,414,2018-10-31 09:22:41.613,2018-10-31 09:29:36.026,473.0,40.721101,-73.991925,268.0,40.719105,-73.999733,1,1993,2,2,"[0.0, 1.0]"
999997,1404,2018-04-10 18:53:09.395,2018-04-10 19:16:34.059,314.0,40.693830,-73.990539,282.0,40.707645,-73.968415,1,1992,2,1,"[0.0, 1.0]"
999998,2293,2018-07-20 18:12:00.797,2018-07-20 18:50:14.762,459.0,40.746745,-74.007756,390.0,40.692216,-73.984284,1,1987,2,4,"[0.0, 1.0]"


In [23]:
small_subset = features_df
train_set, test_set = train_test_split(
    small_subset,
    test_size=0.33, 
    random_state=42, 
    stratify=small_subset[label_column]
)
test_set

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday,usertype_one_hot
533157,361,2018-11-05 19:24:12.765,2018-11-05 19:30:14.170,3417.0,40.679577,-73.978550,3368.0,40.672816,-73.983524,1,1969,0,0,"[0.0, 1.0]"
328648,2176,2018-08-26 16:08:31.129,2018-08-26 16:44:47.831,3587.0,40.668674,-73.961815,414.0,40.702819,-73.987658,0,1992,2,6,"[1.0, 0.0]"
86392,1183,2018-10-15 20:24:34.114,2018-10-15 20:44:17.970,483.0,40.732233,-73.988900,238.0,40.736197,-74.008592,0,1986,1,0,"[1.0, 0.0]"
614082,758,2018-05-19 18:38:55.079,2018-05-19 18:51:33.268,520.0,40.759923,-73.976485,3285.0,40.788390,-73.974700,1,1988,2,5,"[0.0, 1.0]"
548113,597,2018-02-24 11:56:55.329,2018-02-24 12:06:52.941,533.0,40.752996,-73.987216,441.0,40.756014,-73.967416,1,1969,0,5,"[0.0, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362271,49392,2018-06-27 11:57:22.825,2018-06-28 01:40:35.002,3160.0,40.778968,-73.973747,3302.0,40.796935,-73.964341,0,1969,0,2,"[1.0, 0.0]"
1093,1614,2018-10-19 17:08:59.404,2018-10-19 17:35:54.195,3258.0,40.750182,-74.002184,347.0,40.728846,-74.008591,0,2000,1,4,"[1.0, 0.0]"
941967,541,2018-05-18 18:03:18.323,2018-05-18 18:12:19.552,3129.0,40.751102,-73.940737,3613.0,40.745038,-73.957539,1,1983,1,4,"[0.0, 1.0]"
719799,481,2018-07-12 08:05:09.710,2018-07-12 08:13:10.936,3263.0,40.729515,-73.990753,491.0,40.740964,-73.986022,1,1996,1,3,"[0.0, 1.0]"


In [52]:
from keras import Model
from keras.layers import concatenate, Dense, Input


input_layer = Input(shape=(8,))

hidden_1 = Dense(80)(input_layer)
hidden_2 = Dense(40)(hidden_1)
hidden_3 = Dense(20)(hidden_2)

classification = Dense(2, activation='softmax')(hidden_3)

model = Model(inputs=[input_layer], outputs=[classification])

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 80)                720       
_________________________________________________________________
dense_11 (Dense)             (None, 40)                3240      
_________________________________________________________________
dense_12 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 42        
Total params: 4,822
Trainable params: 4,822
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
from keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor='val_accuracy', patience=5)]

train_set_labels = np.array(train_set['usertype_one_hot'].tolist())

model.fit(
    train_set[feature_columns].values, 
    train_set_labels, 
    batch_size=128, 
    epochs=100, 
    validation_split=0.3, 
    callbacks=callbacks
)

Train on 468999 samples, validate on 201001 samples
Epoch 1/100
468999/468999 [==============================] - 6s 12us/step - loss: 0.5569 - accuracy: 0.7780 - val_loss: 0.5390 - val_accuracy: 0.7604
Epoch 2/100
468999/468999 [==============================] - 5s 12us/step - loss: 0.5580 - accuracy: 0.7786 - val_loss: 0.4921 - val_accuracy: 0.8077
Epoch 3/100
468999/468999 [==============================] - 5s 12us/step - loss: 0.5304 - accuracy: 0.7914 - val_loss: 0.5180 - val_accuracy: 0.7482
Epoch 4/100
468999/468999 [==============================] - 6s 12us/step - loss: 0.5341 - accuracy: 0.7803 - val_loss: 0.5128 - val_accuracy: 0.7984
Epoch 5/100
468999/468999 [==============================] - 6s 12us/step - loss: 0.5405 - accuracy: 0.7840 - val_loss: 0.5506 - val_accuracy: 0.7540
Epoch 6/100
468999/468999 [==============================] - 6s 12us/step - loss: 0.5338 - accuracy: 0.7837 - val_loss: 0.6218 - val_accuracy: 0.7005
Epoch 7/100
468999/468999 [=====================

In [58]:
test_set_labels = np.array(test_set[usertype_one_hot].tolist())
model.evaluate([test_set[feature_columns]], test_set_labels, batch_size=128)

330000/330000 [==============================] - 1s 4us/step


[0.4918281561793703, 0.8074060678482056]

In [60]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_set[feature_columns])

print(len(test_set_labels))
print(y_pred)

y_true = [np.argmax(y, axis=None, out=None) for y in test_set_labels]
y_pred = [np.argmax(y, axis=None, out=None) for y in y_pred]

print(classification_report(y_true, y_pred))

330000
[[0.5149544  0.48504558]
 [0.5120308  0.48796922]
 [0.36459616 0.6354039 ]
 ...
 [0.37350443 0.6264956 ]
 [0.31970716 0.68029284]
 [0.25766847 0.74233156]]
              precision    recall  f1-score   support

           0       0.84      0.75      0.80    165000
           1       0.78      0.86      0.82    165000

    accuracy                           0.81    330000
   macro avg       0.81      0.81      0.81    330000
weighted avg       0.81      0.81      0.81    330000



In [28]:
print(len(train_set_labels))
print(len(train_set[feature_columns].values))

670000
670000


In [32]:
reduced_train_set = train_set.groupby('usertype').apply(lambda x: x.sample(2000))
reduced_train_set = reduced_train_set.reset_index(level=[0,1], drop=True)

In [33]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(
    reduced_train_set[feature_columns], 
    reduced_train_set[label_column]
)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [34]:
predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.8421960472439253
[0 1]
['Customer' 'Subscriber']
              precision    recall  f1-score   support

           0       0.93      0.69      0.80    165000
           1       0.76      0.95      0.84    165000

    accuracy                           0.82    330000
   macro avg       0.84      0.82      0.82    330000
weighted avg       0.84      0.82      0.82    330000

